# <center>A modified visual system simulation model improves the robustness of white box adversarial attack (VoneNet)</center>
# <center> Team Member: Shilong Wu, Yinglong Li, Jingze Liu, Yuqi Lei</center>

# <center>Import Library</center>

In [ ]:
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
import cv2
import seaborn
seaborn.set_style('darkgrid')
import shutil
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization,ReLU
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import time
from sklearn.metrics import f1_score
import scipy.stats as stats
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# <center>EfficientNet_B0_1024 Model</center>

In [ ]:
def EfficientNet_B0_Dense1024(img_size, lr, class_count):  
    img_shape=(img_size[0], img_size[1], 3)
    base_model=tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max')
    base_model.trainable=True
    x=base_model.output
    x=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
    x=Dense(1024,activation='relu')(x)
    output=Dense(class_count, activation='sigmoid')(x)
    model=Model(inputs=base_model.input, outputs=output)
    model.compile(Adamax(learning_rate=lr), loss='binary_crossentropy', metrics=['accuracy']) 
    return model

# <center>Basic VoneNet</center>

In [ ]:
def gabor_kernel(frequency, sigma_x, sigma_y, theta=0, offset=0, ks=61):
    w = ks // 2
    grid_val = tf.range(-w, w+1, dtype=tf.float32)
    x, y = tf.meshgrid(grid_val, grid_val)
    rotx = x * np.cos(theta) + y * np.sin(theta)
    roty = -x * np.sin(theta) + y * np.cos(theta)
    g = tf.exp(-0.5 * (rotx ** 2 / sigma_x ** 2 + roty ** 2 / sigma_y ** 2))
    g /= 2 * np.pi * sigma_x * sigma_y
    g *= tf.cos(2 * np.pi * frequency * rotx + offset)

    return g


class Identity(layers.Layer):
    def call(self, inputs):
        return inputs


class GFB(layers.Layer):
    def __init__(self, in_channels, out_channels, kernel_size, stride=4):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride

        self.weight = tf.Variable(tf.zeros(
            (kernel_size, kernel_size, in_channels, out_channels)), trainable=False)

    def call(self, inputs):
        return tf.nn.conv2d(inputs, self.weight, strides=[1, self.stride, self.stride, 1], padding='SAME')

    def initialize(self, sf, theta, sigx, sigy, phase):
        random_channel = tf.random.uniform(
            (self.out_channels,), minval=0, maxval=self.in_channels, dtype=tf.int32)
#         for i in range(self.out_channels):
#             self.weight[i, random_channel[i]] = gabor_kernel(frequency=sf[i], sigma_x=sigx[i], sigma_y=sigy[i],
#                                                              theta=theta[i], offset=phase[i], ks=self.kernel_size)
#         self.weight.assign(self.weight)
        for i in range(self.out_channels):
            gabor = gabor_kernel(frequency=sf[i], sigma_x=sigx[i], sigma_y=sigy[i],
                                     theta=theta[i], offset=phase[i], ks=self.kernel_size)

            # 使用scatter_nd_update更新权重
#             tensor_i = tf.constant(i, dtype=tf.int32)  # 将 i 转换为张量
#             indices = tf.constant([[tensor_i, random_channel[i]]], dtype=tf.int32)
# #             indices = tf.constant([[i, random_channel[i]]], dtype=tf.int32)
#             updates = tf.expand_dims(gabor, axis=0)
#             self.weight = tf.tensor_scatter_nd_update(self.weight, indices, updates)
#             tensor_i = tf.constant(i, dtype=tf.int32)  # 将 i 转换为张量
#             tensor_channel = tf.constant(random_channel[i], dtype=tf.int32)  # 将 random_channel[i] 转换为张量
#             indices = tf.expand_dims(tf.stack([tensor_i, tensor_channel]), axis=0)  # 堆叠张量并增加维度
#             updates = tf.expand_dims(gabor, axis=0)
#             self.weight = tf.tensor_scatter_nd_update(self.weight, indices, updates)

            kernel = gabor_kernel(frequency=sf[i], sigma_x=sigx[i], sigma_y=sigy[i],
                                  theta=theta[i], offset=phase[i], ks=self.kernel_size)
            self.weight[:, :, random_channel[i], i].assign(kernel)

class VOneBlock(layers.Layer):
    def __init__(self, sf, theta, sigx, sigy, phase,
                 k_exc=25, noise_mode=None, noise_scale=1, noise_level=1,
                 simple_channels=128, complex_channels=128, ksize=25, stride=4, input_size=224):
        super().__init__()

        self.in_channels = 3
        self.simple_channels = simple_channels
        self.complex_channels = complex_channels
        self.out_channels = simple_channels + complex_channels
        self.stride = stride
        self.input_size = input_size

        self.sf = sf
        self.theta = theta
        self.sigx = sigx
        self.sigy = sigy
        self.phase = phase
        self.k_exc = k_exc

        self.set_noise_mode(noise_mode, noise_scale, noise_level)
        self.fixed_noise = None

        self.simple_conv_q0 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q1 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q0.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase)
        self.simple_conv_q1.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi / 2)

        self.simple = layers.ReLU()
        self.complex = Identity()
        self.gabors = Identity()
        self.noise = layers.ReLU()
        self.output_layer = Identity()

    def call(self, inputs):
        x = self.gabors_f(inputs)
        x = self.noise_f(x)
        x = self.output_layer(x)
        return x

    def gabors_f(self, inputs):
        s_q0 = self.simple_conv_q0(inputs)
        s_q1 = self.simple_conv_q1(inputs)
        c = self.complex(tf.math.sqrt(s_q0[:, :, :, self.simple_channels:] ** 2 +
                                      s_q1[:, :, :, self.simple_channels:] ** 2) / np.sqrt(2))
        s = self.simple(s_q0[:, :, :, :self.simple_channels])

        return self.gabors(self.k_exc * tf.concat((s, c), axis=-1))

    def noise_f(self, inputs):
        x = inputs
        if self.noise_mode == 'neuronal':
            eps = 10e-5
            x *= self.noise_scale
            x += self.noise_level
            if self.fixed_noise is not None:
                x += self.fixed_noise * tf.math.sqrt(tf.nn.relu(x) + eps)
            else:
                x += tf.random.normal(tf.shape(x), stddev=1) * \
                    tf.math.sqrt(tf.nn.relu(x) + eps)
            x -= self.noise_level
            x /= self.noise_scale
        if self.noise_mode == 'gaussian':
            if self.fixed_noise is not None:
                x += self.fixed_noise * self.noise_scale
            else:
                x += tf.random.normal(tf.shape(x), stddev=1) * self.noise_scale
        return self.noise(x)

    def set_noise_mode(self, noise_mode=None, noise_scale=1, noise_level=1):
        self.noise_mode = noise_mode
        self.noise_scale = noise_scale
        self.noise_level = noise_level

    def fix_noise(self, batch_size=256, seed=None):
        if seed:
            tf.random.set_seed(seed)
        noise_mean = tf.zeros((batch_size, int(
            self.input_size / self.stride), int(self.input_size / self.stride), self.out_channels))
        if self.noise_mode:
            self.fixed_noise = tf.random.normal(noise_mean.shape, stddev=1)

    def unfix_noise(self):
        self.fixed_noise = None
        
    
def sample_dist(hist, bins, ns, scale='linear'):
    rand_sample = np.random.rand(ns)
    if scale == 'linear':
        rand_sample = np.interp(
            rand_sample, np.hstack(([0], hist.cumsum())), bins)
    elif scale == 'log2':
        rand_sample = np.interp(rand_sample, np.hstack(
            ([0], hist.cumsum())), np.log2(bins))
        rand_sample = 2**rand_sample
    elif scale == 'log10':
        rand_sample = np.interp(rand_sample, np.hstack(
            ([0], hist.cumsum())), np.log10(bins))
        rand_sample = 10**rand_sample
    return rand_sample


def generate_gabor_param(features, seed=0, rand_flag=False, sf_corr=0, sf_max=9, sf_min=0):
    # Generates random sample
    np.random.seed(seed)

    phase_bins = np.array([0, 360])
    phase_dist = np.array([1])

    if rand_flag:
        print('Uniform gabor parameters')
        ori_bins = np.array([0, 180])
        ori_dist = np.array([1])

        nx_bins = np.array([0.1, 10**0.2])
        nx_dist = np.array([1])

        ny_bins = np.array([0.1, 10**0.2])
        ny_dist = np.array([1])

        # sf_bins = np.array([0.5, 8])
        # sf_dist = np.array([1])

        sf_bins = np.array([0.5, 0.7, 1.0, 1.4, 2.0, 2.8, 4.0, 5.6, 8])
        sf_dist = np.array([1,  1,  1, 1, 1, 1, 1, 1])

        sfmax_ind = np.where(sf_bins < sf_max)[0][-1]
        sfmin_ind = np.where(sf_bins >= sf_min)[0][0]

        sf_bins = sf_bins[sfmin_ind:sfmax_ind+1]
        sf_dist = sf_dist[sfmin_ind:sfmax_ind]

        sf_dist = sf_dist / sf_dist.sum()
    else:
        print('Neuronal distributions gabor parameters')
        # DeValois 1982a
        ori_bins = np.array([-22.5, 22.5, 67.5, 112.5, 157.5])
        ori_dist = np.array([66, 49, 77, 54])
        ori_dist = ori_dist / ori_dist.sum()

        # Schiller 1976
        cov_mat = np.array([[1, sf_corr], [sf_corr, 1]])

        # Ringach 2002b
        nx_bins = np.logspace(-1, 0.2, 6, base=10)
        ny_bins = np.logspace(-1, 0.2, 6, base=10)
        n_joint_dist = np.array([[2.,  0.,  1.,  0.,  0.],
                                 [8.,  9.,  4.,  1.,  0.],
                                 [1.,  2., 19., 17.,  3.],
                                 [0.,  0.,  1.,  7.,  4.],
                                 [0.,  0.,  0.,  0.,  0.]])
        n_joint_dist = n_joint_dist / n_joint_dist.sum()
        nx_dist = n_joint_dist.sum(axis=1)
        nx_dist = nx_dist / nx_dist.sum()
        ny_dist_marg = n_joint_dist / n_joint_dist.sum(axis=1, keepdims=True)

        # DeValois 1982b
        sf_bins = np.array([0.5, 0.7, 1.0, 1.4, 2.0, 2.8, 4.0, 5.6, 8])
        sf_dist = np.array([4,  4,  8, 25, 32, 26, 28, 12])

        sfmax_ind = np.where(sf_bins <= sf_max)[0][-1]
        sfmin_ind = np.where(sf_bins >= sf_min)[0][0]

        sf_bins = sf_bins[sfmin_ind:sfmax_ind+1]
        sf_dist = sf_dist[sfmin_ind:sfmax_ind]

        sf_dist = sf_dist / sf_dist.sum()

    phase = sample_dist(phase_dist, phase_bins, features)
    ori = sample_dist(ori_dist, ori_bins, features)
    ori[ori < 0] = ori[ori < 0] + 180

    if rand_flag:
        sf = sample_dist(sf_dist, sf_bins, features, scale='log2')
        nx = sample_dist(nx_dist, nx_bins, features, scale='log10')
        ny = sample_dist(ny_dist, ny_bins, features, scale='log10')
    else:

        samps = np.random.multivariate_normal([0, 0], cov_mat, features)
        samps_cdf = stats.norm.cdf(samps)

        nx = np.interp(samps_cdf[:, 0], np.hstack(
            ([0], nx_dist.cumsum())), np.log10(nx_bins))
        nx = 10**nx

        ny_samp = np.random.rand(features)
        ny = np.zeros(features)
        for samp_ind, nx_samp in enumerate(nx):
            bin_id = np.argwhere(nx_bins < nx_samp)[-1]
            ny[samp_ind] = np.interp(ny_samp[samp_ind], np.hstack(([0], ny_dist_marg[bin_id, :].cumsum())),
                                     np.log10(ny_bins))
        ny = 10**ny

        sf = np.interp(samps_cdf[:, 1], np.hstack(
            ([0], sf_dist.cumsum())), np.log2(sf_bins))
        sf = 2**sf

    return sf, ori, phase, nx, ny


def VOneNet(sf_corr=0.75, sf_max=9, sf_min=0, rand_param=False, gabor_seed=0,
            simple_channels=256, complex_channels=256,
            noise_mode='neuronal', noise_scale=0.35, noise_level=0.07, k_exc=25,
            model_arch='resnet50', image_size=300, visual_degrees=8, ksize=25, stride=4, class_count=14):

    out_channels = simple_channels + complex_channels

    sf, theta, phase, nx, ny = generate_gabor_param(
        out_channels, gabor_seed, rand_param, sf_corr, sf_max, sf_min)

    gabor_params = {'simple_channels': simple_channels, 'complex_channels': complex_channels, 'rand_param': rand_param,
                    'gabor_seed': gabor_seed, 'sf_max': sf_max, 'sf_corr': sf_corr, 'sf': sf.copy(),
                    'theta': theta.copy(), 'phase': phase.copy(), 'nx': nx.copy(), 'ny': ny.copy()}
    arch_params = {'k_exc': k_exc, 'arch': model_arch,
                   'ksize': ksize, 'stride': stride}

    # Conversions
    ppd = image_size / visual_degrees

    sf = sf / ppd
    sigx = nx / sf
    sigy = ny / sf
    theta = theta/180 * np.pi
    phase = phase / 180 * np.pi

    vone_block = VOneBlock(sf=sf, theta=theta, sigx=sigx, sigy=sigy, phase=phase,
                           k_exc=k_exc, noise_mode=noise_mode, noise_scale=noise_scale, noise_level=noise_level,
                           simple_channels=simple_channels, complex_channels=complex_channels,
                           ksize=ksize, stride=stride, input_size=image_size)
    
    
    
    
    img_shape=(image_size,image_size,3)
#     model_back_end=tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max')
    
#     inputs = tf.keras.Input(shape=(image_size, image_size, 3))
#     x = vone_block(inputs)
    
    
#     if model_arch:
    bottleneck = Conv2D(3, kernel_size=1, strides=1, use_bias=False,kernel_initializer=tf.keras.initializers.HeNormal())

#         # if model_arch.lower() == 'resnet50':
#         #     print('Model: ', 'VOneResnet50')
#         #     model_back_end = ResNetBackEnd(
#         #         block=Bottleneck, layers=[3, 4, 6, 3])
#         # elif model_arch.lower() == 'alexnet':
#         #     print('Model: ', 'VOneAlexNet')
#         #     model_back_end = AlexNetBackEnd()
#         # elif model_arch.lower() == 'cornets':
#         #     print('Model: ', 'VOneCORnet-S')
#         #     model_back_end = CORnetSBackEnd()
#         # TODO: change model_back_end to our block

    inputs = tf.keras.Input(shape=(image_size, image_size, 3))
    x = vone_block(inputs)
    x = bottleneck(x)
    
    
    # 获取模型后端的输入张量
#     backend_input = model_back_end.input

#     print(x.shape)
#     assert(False)
    
    
    model_back_end=tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, weights="imagenet",input_shape=x.shape[1:], pooling='max')
    model_back_end.trainable = True
#     print(backend_input.shape)

    
    # 用 bottleneck 的输出替换模型后端的输入张量
    backend_output = model_back_end(x)
    
    
    x=backend_output
    x=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
    x=Dense(1024,activation='relu')(x)
    outputs=Dense(class_count, activation='sigmoid')(x)

    
#     outputs = model_back_end.output
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(Adamax(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy']) 
#     else:
#         print('Model: ', 'VOneNet')
#         model = vone_block

    model.image_size = image_size
    model.visual_degrees = visual_degrees
    model.gabor_parms = gabor_params
    model.arch_params = arch_params

    return model





# <center>VOneNet_with_Only_Simple_GFBs</center>

In [ ]:

class VOneBlock_Only_Simple_GFBs(layers.Layer):
    def __init__(self, sf, theta, sigx, sigy, phase,
                 k_exc=25, noise_mode=None, noise_scale=1, noise_level=1,
                 simple_channels=128, complex_channels=128, ksize=25, stride=4, input_size=224):
        super().__init__()

        self.in_channels = 3
        self.simple_channels = simple_channels
        self.complex_channels = complex_channels
        self.out_channels = simple_channels + complex_channels
        self.stride = stride
        self.input_size = input_size

        self.sf = sf
        self.theta = theta
        self.sigx = sigx
        self.sigy = sigy
        self.phase = phase
        self.k_exc = k_exc

        self.set_noise_mode(noise_mode, noise_scale, noise_level)
        self.fixed_noise = None

        self.simple_conv_q0 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q1 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        
        self.simple_conv_q0.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase)
        self.simple_conv_q1.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi / 2)

        self.simple = layers.ReLU()
        self.complex = Identity()
        self.gabors = Identity()
        self.noise = layers.ReLU()
        self.output_layer = Identity()

    def call(self, inputs):
        x = self.gabors_f(inputs)
        x = self.noise_f(x)
        x = self.output_layer(x)
        return x

    def gabors_f(self, inputs):
        s_q0 = self.simple_conv_q0(inputs)
        s_q1 = self.simple_conv_q1(inputs)
        c = self.complex(tf.math.sqrt(s_q0[:, :, :, self.simple_channels:] ** 2 +
                                      s_q1[:, :, :, self.simple_channels:] ** 2 ) / np.sqrt(2))
        s = self.simple(s_q0[:, :, :, :self.simple_channels])

        return self.gabors(self.k_exc * s)

    def noise_f(self, inputs):
        x = inputs
        if self.noise_mode == 'neuronal':
            eps = 10e-5
            x *= self.noise_scale
            x += self.noise_level
            if self.fixed_noise is not None:
                x += self.fixed_noise * tf.math.sqrt(tf.nn.relu(x) + eps)
            else:
                x += tf.random.normal(tf.shape(x), stddev=1) * \
                    tf.math.sqrt(tf.nn.relu(x) + eps)
            x -= self.noise_level
            x /= self.noise_scale
        if self.noise_mode == 'gaussian':
            if self.fixed_noise is not None:
                x += self.fixed_noise * self.noise_scale
            else:
                x += tf.random.normal(tf.shape(x), stddev=1) * self.noise_scale
        return self.noise(x)

    def set_noise_mode(self, noise_mode=None, noise_scale=1, noise_level=1):
        self.noise_mode = noise_mode
        self.noise_scale = noise_scale
        self.noise_level = noise_level

    def fix_noise(self, batch_size=256, seed=None):
        if seed:
            tf.random.set_seed(seed)
        noise_mean = tf.zeros((batch_size, int(
            self.input_size / self.stride), int(self.input_size / self.stride), self.out_channels))
        if self.noise_mode:
            self.fixed_noise = tf.random.normal(noise_mean.shape, stddev=1)

    def unfix_noise(self):
        self.fixed_noise = None

        
        
        
        
        
        


def VOneNet_with_Only_Simple_GFBs(sf_corr=0.75, sf_max=9, sf_min=0, rand_param=False, gabor_seed=0,
            simple_channels=256, complex_channels=256,
            noise_mode='neuronal', noise_scale=0.35, noise_level=0.07, k_exc=25,
            model_arch='resnet50', image_size=300, visual_degrees=8, ksize=25, stride=4, class_count=14):

    out_channels = simple_channels + complex_channels

    sf, theta, phase, nx, ny = generate_gabor_param(
        out_channels, gabor_seed, rand_param, sf_corr, sf_max, sf_min)

    gabor_params = {'simple_channels': simple_channels, 'complex_channels': complex_channels, 'rand_param': rand_param,
                    'gabor_seed': gabor_seed, 'sf_max': sf_max, 'sf_corr': sf_corr, 'sf': sf.copy(),
                    'theta': theta.copy(), 'phase': phase.copy(), 'nx': nx.copy(), 'ny': ny.copy()}
    arch_params = {'k_exc': k_exc, 'arch': model_arch,
                   'ksize': ksize, 'stride': stride}

    # Conversions
    ppd = image_size / visual_degrees

    sf = sf / ppd
    sigx = nx / sf
    sigy = ny / sf
    theta = theta/180 * np.pi
    phase = phase / 180 * np.pi

    vone_block = VOneBlock_Only_Simple_GFBs(sf=sf, theta=theta, sigx=sigx, sigy=sigy, phase=phase,
                           k_exc=k_exc, noise_mode=noise_mode, noise_scale=noise_scale, noise_level=noise_level,
                           simple_channels=simple_channels, complex_channels=complex_channels,
                           ksize=ksize, stride=stride, input_size=image_size)
    
    
    
    
    img_shape=(image_size,image_size,3)
#     model_back_end=tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max')
    
#     inputs = tf.keras.Input(shape=(image_size, image_size, 3))
#     x = vone_block(inputs)
    
    
#     if model_arch:
    bottleneck = Conv2D(3, kernel_size=1, strides=1, use_bias=False,kernel_initializer=tf.keras.initializers.HeNormal())

#         # if model_arch.lower() == 'resnet50':
#         #     print('Model: ', 'VOneResnet50')
#         #     model_back_end = ResNetBackEnd(
#         #         block=Bottleneck, layers=[3, 4, 6, 3])
#         # elif model_arch.lower() == 'alexnet':
#         #     print('Model: ', 'VOneAlexNet')
#         #     model_back_end = AlexNetBackEnd()
#         # elif model_arch.lower() == 'cornets':
#         #     print('Model: ', 'VOneCORnet-S')
#         #     model_back_end = CORnetSBackEnd()
#         # TODO: change model_back_end to our block

    inputs = tf.keras.Input(shape=(image_size, image_size, 3))
    x = vone_block(inputs)
    x = bottleneck(x)
    
    
    # 获取模型后端的输入张量
#     backend_input = model_back_end.input

#     print(x.shape)
#     assert(False)
    
    
    model_back_end=tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, weights="imagenet",input_shape=x.shape[1:], pooling='max')
    model_back_end.trainable = True
#     print(backend_input.shape)

    
    # 用 bottleneck 的输出替换模型后端的输入张量
    backend_output = model_back_end(x)
    
    
    x=backend_output
    x=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
    x=Dense(1024,activation='relu')(x)
    outputs=Dense(class_count, activation='sigmoid')(x)

    
#     outputs = model_back_end.output
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(Adamax(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy']) 
#     else:
#         print('Model: ', 'VOneNet')
#         model = vone_block

    model.image_size = image_size
    model.visual_degrees = visual_degrees
    model.gabor_parms = gabor_params
    model.arch_params = arch_params

    return model


# <center>VOneNet_with_4_GFBs</center>

In [ ]:
    
class VOneBlock_Four_GFBs(layers.Layer):
    def __init__(self, sf, theta, sigx, sigy, phase,
                 k_exc=25, noise_mode=None, noise_scale=1, noise_level=1,
                 simple_channels=128, complex_channels=128, ksize=25, stride=4, input_size=224):
        super().__init__()

        self.in_channels = 3
        self.simple_channels = simple_channels
        self.complex_channels = complex_channels
        self.out_channels = simple_channels + complex_channels
        self.stride = stride
        self.input_size = input_size

        self.sf = sf
        self.theta = theta
        self.sigx = sigx
        self.sigy = sigy
        self.phase = phase
        self.k_exc = k_exc

        self.set_noise_mode(noise_mode, noise_scale, noise_level)
        self.fixed_noise = None

        self.simple_conv_q0 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q1 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q2 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q3 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        
        self.simple_conv_q0.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase)
        self.simple_conv_q1.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi / 4)
        self.simple_conv_q2.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi / 2)
        self.simple_conv_q3.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi * 3 / 4)

        self.simple = layers.ReLU()
        self.complex = Identity()
        self.gabors = Identity()
        self.noise = layers.ReLU()
        self.output_layer = Identity()

    def call(self, inputs):
        x = self.gabors_f(inputs)
        x = self.noise_f(x)
        x = self.output_layer(x)
        return x

    def gabors_f(self, inputs):
        s_q0 = self.simple_conv_q0(inputs)
        s_q1 = self.simple_conv_q1(inputs)
        s_q2 = self.simple_conv_q2(inputs)
        s_q3 = self.simple_conv_q3(inputs)
        c = self.complex(tf.math.sqrt(s_q0[:, :, :, self.simple_channels:] ** 2 +
                                      s_q1[:, :, :, self.simple_channels:] ** 2 +
                                      s_q2[:, :, :, self.simple_channels:] ** 2 +
                                      s_q3[:, :, :, self.simple_channels:] ** 2) / np.sqrt(4))
        s = self.simple(s_q0[:, :, :, :self.simple_channels])

        return self.gabors(self.k_exc * tf.concat((s, c), axis=-1))

    def noise_f(self, inputs):
        x = inputs
        if self.noise_mode == 'neuronal':
            eps = 10e-5
            x *= self.noise_scale
            x += self.noise_level
            if self.fixed_noise is not None:
                x += self.fixed_noise * tf.math.sqrt(tf.nn.relu(x) + eps)
            else:
                x += tf.random.normal(tf.shape(x), stddev=1) * \
                    tf.math.sqrt(tf.nn.relu(x) + eps)
            x -= self.noise_level
            x /= self.noise_scale
        if self.noise_mode == 'gaussian':
            if self.fixed_noise is not None:
                x += self.fixed_noise * self.noise_scale
            else:
                x += tf.random.normal(tf.shape(x), stddev=1) * self.noise_scale
        return self.noise(x)

    def set_noise_mode(self, noise_mode=None, noise_scale=1, noise_level=1):
        self.noise_mode = noise_mode
        self.noise_scale = noise_scale
        self.noise_level = noise_level

    def fix_noise(self, batch_size=256, seed=None):
        if seed:
            tf.random.set_seed(seed)
        noise_mean = tf.zeros((batch_size, int(
            self.input_size / self.stride), int(self.input_size / self.stride), self.out_channels))
        if self.noise_mode:
            self.fixed_noise = tf.random.normal(noise_mean.shape, stddev=1)

    def unfix_noise(self):
        self.fixed_noise = None

        
        
        
        
        
        


def VOneNet_with_four_GFBs(sf_corr=0.75, sf_max=9, sf_min=0, rand_param=False, gabor_seed=0,
            simple_channels=256, complex_channels=256,
            noise_mode='neuronal', noise_scale=0.35, noise_level=0.07, k_exc=25,
            model_arch='resnet50', image_size=300, visual_degrees=8, ksize=25, stride=4, class_count=14):

    out_channels = simple_channels + complex_channels

    sf, theta, phase, nx, ny = generate_gabor_param(
        out_channels, gabor_seed, rand_param, sf_corr, sf_max, sf_min)

    gabor_params = {'simple_channels': simple_channels, 'complex_channels': complex_channels, 'rand_param': rand_param,
                    'gabor_seed': gabor_seed, 'sf_max': sf_max, 'sf_corr': sf_corr, 'sf': sf.copy(),
                    'theta': theta.copy(), 'phase': phase.copy(), 'nx': nx.copy(), 'ny': ny.copy()}
    arch_params = {'k_exc': k_exc, 'arch': model_arch,
                   'ksize': ksize, 'stride': stride}

    # Conversions
    ppd = image_size / visual_degrees

    sf = sf / ppd
    sigx = nx / sf
    sigy = ny / sf
    theta = theta/180 * np.pi
    phase = phase / 180 * np.pi

    vone_block = VOneBlock_Four_GFBs(sf=sf, theta=theta, sigx=sigx, sigy=sigy, phase=phase,
                           k_exc=k_exc, noise_mode=noise_mode, noise_scale=noise_scale, noise_level=noise_level,
                           simple_channels=simple_channels, complex_channels=complex_channels,
                           ksize=ksize, stride=stride, input_size=image_size)
    
    
    
    
    img_shape=(image_size,image_size,3)
#     model_back_end=tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max')
    
#     inputs = tf.keras.Input(shape=(image_size, image_size, 3))
#     x = vone_block(inputs)
    
    
#     if model_arch:
    bottleneck = Conv2D(3, kernel_size=1, strides=1, use_bias=False,kernel_initializer=tf.keras.initializers.HeNormal())

#         # if model_arch.lower() == 'resnet50':
#         #     print('Model: ', 'VOneResnet50')
#         #     model_back_end = ResNetBackEnd(
#         #         block=Bottleneck, layers=[3, 4, 6, 3])
#         # elif model_arch.lower() == 'alexnet':
#         #     print('Model: ', 'VOneAlexNet')
#         #     model_back_end = AlexNetBackEnd()
#         # elif model_arch.lower() == 'cornets':
#         #     print('Model: ', 'VOneCORnet-S')
#         #     model_back_end = CORnetSBackEnd()
#         # TODO: change model_back_end to our block

    inputs = tf.keras.Input(shape=(image_size, image_size, 3))
    x = vone_block(inputs)
    x = bottleneck(x)
    
    
    # 获取模型后端的输入张量
#     backend_input = model_back_end.input

#     print(x.shape)
#     assert(False)
    
    
    model_back_end=tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, weights="imagenet",input_shape=x.shape[1:], pooling='max')
    model_back_end.trainable = True
#     print(backend_input.shape)

    
    # 用 bottleneck 的输出替换模型后端的输入张量
    backend_output = model_back_end(x)
    
    
    x=backend_output
    x=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
    x=Dense(1024,activation='relu')(x)
    outputs=Dense(class_count, activation='sigmoid')(x)

    
#     outputs = model_back_end.output
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(Adamax(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy']) 
#     else:
#         print('Model: ', 'VOneNet')
#         model = vone_block

    model.image_size = image_size
    model.visual_degrees = visual_degrees
    model.gabor_parms = gabor_params
    model.arch_params = arch_params

    return model


# <center>VOneNet_with_8_GFBs</center>

In [ ]:
  
class VOneBlock_Eight_GFBs(layers.Layer):
    def __init__(self, sf, theta, sigx, sigy, phase,
                 k_exc=25, noise_mode=None, noise_scale=1, noise_level=1,
                 simple_channels=128, complex_channels=128, ksize=25, stride=4, input_size=224):
        super().__init__()

        self.in_channels = 3
        self.simple_channels = simple_channels
        self.complex_channels = complex_channels
        self.out_channels = simple_channels + complex_channels
        self.stride = stride
        self.input_size = input_size

        self.sf = sf
        self.theta = theta
        self.sigx = sigx
        self.sigy = sigy
        self.phase = phase
        self.k_exc = k_exc

        self.set_noise_mode(noise_mode, noise_scale, noise_level)
        self.fixed_noise = None

        self.simple_conv_q0 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q1 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q2 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q3 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        
        self.simple_conv_q4 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q5 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q6 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q7 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        
        self.simple_conv_q0.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase)
        self.simple_conv_q1.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi / 8)
        self.simple_conv_q2.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi * 2 / 8)
        self.simple_conv_q3.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi * 3 / 8)
        self.simple_conv_q4.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi * 4 / 8)
        self.simple_conv_q5.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi * 5 / 8)
        self.simple_conv_q6.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi * 6 / 8)
        self.simple_conv_q7.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi * 7 / 8)

        self.simple = layers.ReLU()
        self.complex = Identity()
        self.gabors = Identity()
        self.noise = layers.ReLU()
        self.output_layer = Identity()

    def call(self, inputs):
        x = self.gabors_f(inputs)
        x = self.noise_f(x)
        x = self.output_layer(x)
        return x

    def gabors_f(self, inputs):
        s_q0 = self.simple_conv_q0(inputs)
        s_q1 = self.simple_conv_q1(inputs)
        s_q2 = self.simple_conv_q2(inputs)
        s_q3 = self.simple_conv_q3(inputs)
        s_q4 = self.simple_conv_q4(inputs)
        s_q5 = self.simple_conv_q5(inputs)
        s_q6 = self.simple_conv_q6(inputs)
        s_q7 = self.simple_conv_q7(inputs)
        c = self.complex(tf.math.sqrt(s_q0[:, :, :, self.simple_channels:] ** 2 +
                                      s_q1[:, :, :, self.simple_channels:] ** 2 +
                                      s_q2[:, :, :, self.simple_channels:] ** 2 +
                                      s_q3[:, :, :, self.simple_channels:] ** 2 +
                                      s_q4[:, :, :, self.simple_channels:] ** 2 +
                                      s_q5[:, :, :, self.simple_channels:] ** 2 +
                                      s_q6[:, :, :, self.simple_channels:] ** 2 +
                                      s_q7[:, :, :, self.simple_channels:] ** 2 ) / np.sqrt(8))
        s = self.simple(s_q0[:, :, :, :self.simple_channels])

        return self.gabors(self.k_exc * tf.concat((s, c), axis=-1))

    def noise_f(self, inputs):
        x = inputs
        if self.noise_mode == 'neuronal':
            eps = 10e-5
            x *= self.noise_scale
            x += self.noise_level
            if self.fixed_noise is not None:
                x += self.fixed_noise * tf.math.sqrt(tf.nn.relu(x) + eps)
            else:
                x += tf.random.normal(tf.shape(x), stddev=1) * \
                    tf.math.sqrt(tf.nn.relu(x) + eps)
            x -= self.noise_level
            x /= self.noise_scale
        if self.noise_mode == 'gaussian':
            if self.fixed_noise is not None:
                x += self.fixed_noise * self.noise_scale
            else:
                x += tf.random.normal(tf.shape(x), stddev=1) * self.noise_scale
        return self.noise(x)

    def set_noise_mode(self, noise_mode=None, noise_scale=1, noise_level=1):
        self.noise_mode = noise_mode
        self.noise_scale = noise_scale
        self.noise_level = noise_level

    def fix_noise(self, batch_size=256, seed=None):
        if seed:
            tf.random.set_seed(seed)
        noise_mean = tf.zeros((batch_size, int(
            self.input_size / self.stride), int(self.input_size / self.stride), self.out_channels))
        if self.noise_mode:
            self.fixed_noise = tf.random.normal(noise_mean.shape, stddev=1)

    def unfix_noise(self):
        self.fixed_noise = None

        
        
        
        
        
        


def VOneNet_with_eight_GFBs(sf_corr=0.75, sf_max=9, sf_min=0, rand_param=False, gabor_seed=0,
            simple_channels=256, complex_channels=256,
            noise_mode='neuronal', noise_scale=0.35, noise_level=0.07, k_exc=25,
            model_arch='resnet50', image_size=300, visual_degrees=8, ksize=25, stride=4, class_count=14):

    out_channels = simple_channels + complex_channels

    sf, theta, phase, nx, ny = generate_gabor_param(
        out_channels, gabor_seed, rand_param, sf_corr, sf_max, sf_min)

    gabor_params = {'simple_channels': simple_channels, 'complex_channels': complex_channels, 'rand_param': rand_param,
                    'gabor_seed': gabor_seed, 'sf_max': sf_max, 'sf_corr': sf_corr, 'sf': sf.copy(),
                    'theta': theta.copy(), 'phase': phase.copy(), 'nx': nx.copy(), 'ny': ny.copy()}
    arch_params = {'k_exc': k_exc, 'arch': model_arch,
                   'ksize': ksize, 'stride': stride}

    # Conversions
    ppd = image_size / visual_degrees

    sf = sf / ppd
    sigx = nx / sf
    sigy = ny / sf
    theta = theta/180 * np.pi
    phase = phase / 180 * np.pi

    vone_block = VOneBlock_Eight_GFBs(sf=sf, theta=theta, sigx=sigx, sigy=sigy, phase=phase,
                           k_exc=k_exc, noise_mode=noise_mode, noise_scale=noise_scale, noise_level=noise_level,
                           simple_channels=simple_channels, complex_channels=complex_channels,
                           ksize=ksize, stride=stride, input_size=image_size)
    
    
    
    
    img_shape=(image_size,image_size,3)
#     model_back_end=tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max')
    
#     inputs = tf.keras.Input(shape=(image_size, image_size, 3))
#     x = vone_block(inputs)
    
    
#     if model_arch:
    bottleneck = Conv2D(3, kernel_size=1, strides=1, use_bias=False,kernel_initializer=tf.keras.initializers.HeNormal())

#         # if model_arch.lower() == 'resnet50':
#         #     print('Model: ', 'VOneResnet50')
#         #     model_back_end = ResNetBackEnd(
#         #         block=Bottleneck, layers=[3, 4, 6, 3])
#         # elif model_arch.lower() == 'alexnet':
#         #     print('Model: ', 'VOneAlexNet')
#         #     model_back_end = AlexNetBackEnd()
#         # elif model_arch.lower() == 'cornets':
#         #     print('Model: ', 'VOneCORnet-S')
#         #     model_back_end = CORnetSBackEnd()
#         # TODO: change model_back_end to our block

    inputs = tf.keras.Input(shape=(image_size, image_size, 3))
    x = vone_block(inputs)
    x = bottleneck(x)
    
    
    # 获取模型后端的输入张量
#     backend_input = model_back_end.input

#     print(x.shape)
#     assert(False)
    
    
    model_back_end=tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, weights="imagenet",input_shape=x.shape[1:], pooling='max')
    model_back_end.trainable = True
#     print(backend_input.shape)

    
    # 用 bottleneck 的输出替换模型后端的输入张量
    backend_output = model_back_end(x)
    
    
    x=backend_output
    x=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
    x=Dense(1024,activation='relu')(x)
    outputs=Dense(class_count, activation='sigmoid')(x)

    
#     outputs = model_back_end.output
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(Adamax(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy']) 
#     else:
#         print('Model: ', 'VOneNet')
#         model = vone_block

    model.image_size = image_size
    model.visual_degrees = visual_degrees
    model.gabor_parms = gabor_params
    model.arch_params = arch_params

    return model


# <center>VOneNet_with_16_GFBs</center>

In [ ]:

class VOneBlock_16_GFBs(layers.Layer):
    def __init__(self, sf, theta, sigx, sigy, phase,
                 k_exc=25, noise_mode=None, noise_scale=1, noise_level=1,
                 simple_channels=128, complex_channels=128, ksize=25, stride=4, input_size=224):
        super().__init__()

        self.in_channels = 3
        self.simple_channels = simple_channels
        self.complex_channels = complex_channels
        self.out_channels = simple_channels + complex_channels
        self.stride = stride
        self.input_size = input_size

        self.sf = sf
        self.theta = theta
        self.sigx = sigx
        self.sigy = sigy
        self.phase = phase
        self.k_exc = k_exc

        self.set_noise_mode(noise_mode, noise_scale, noise_level)
        self.fixed_noise = None

        self.simple_conv_q0 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q1 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q2 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q3 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        
        self.simple_conv_q4 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q5 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q6 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q7 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        
        
        self.simple_conv_q8 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q9 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q10 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q11 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        
        self.simple_conv_q12 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q13 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q14 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        self.simple_conv_q15 = GFB(
            self.in_channels, self.out_channels, ksize, stride)
        
        self.simple_conv_q0.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase)
        self.simple_conv_q1.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi / 16)
        self.simple_conv_q2.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi * 2 / 16)
        self.simple_conv_q3.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi * 3 / 16)
        self.simple_conv_q4.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi * 4 / 16)
        self.simple_conv_q5.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi * 5 / 16)
        self.simple_conv_q6.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi * 6 / 16)
        self.simple_conv_q7.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi * 7 / 16)
        
        
        
        self.simple_conv_q8.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi * 8 / 16)
        self.simple_conv_q9.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi * 9 / 16)
        self.simple_conv_q10.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi * 10 / 16)
        self.simple_conv_q11.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi * 11 / 16)
        self.simple_conv_q12.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi * 12 / 16)
        self.simple_conv_q13.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi * 13 / 16)
        self.simple_conv_q14.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi * 14 / 16)
        self.simple_conv_q15.initialize(sf=self.sf, theta=self.theta, sigx=self.sigx, sigy=self.sigy,
                                       phase=self.phase + np.pi * 15 / 16)

        self.simple = layers.ReLU()
        self.complex = Identity()
        self.gabors = Identity()
        self.noise = layers.ReLU()
        self.output_layer = Identity()

    def call(self, inputs):
        x = self.gabors_f(inputs)
        x = self.noise_f(x)
        x = self.output_layer(x)
        return x

    def gabors_f(self, inputs):
        s_q0 = self.simple_conv_q0(inputs)
        s_q1 = self.simple_conv_q1(inputs)
        s_q2 = self.simple_conv_q2(inputs)
        s_q3 = self.simple_conv_q3(inputs)
        s_q4 = self.simple_conv_q4(inputs)
        s_q5 = self.simple_conv_q5(inputs)
        s_q6 = self.simple_conv_q6(inputs)
        s_q7 = self.simple_conv_q7(inputs)
        
        
        s_q8 = self.simple_conv_q8(inputs)
        s_q9 = self.simple_conv_q9(inputs)
        s_q10 = self.simple_conv_q10(inputs)
        s_q11 = self.simple_conv_q11(inputs)
        s_q12 = self.simple_conv_q12(inputs)
        s_q13 = self.simple_conv_q13(inputs)
        s_q14 = self.simple_conv_q14(inputs)
        s_q15 = self.simple_conv_q15(inputs)
        
        c = self.complex(tf.math.sqrt(s_q0[:, :, :, self.simple_channels:] ** 2 +
                                      s_q1[:, :, :, self.simple_channels:] ** 2 +
                                      s_q2[:, :, :, self.simple_channels:] ** 2 +
                                      s_q3[:, :, :, self.simple_channels:] ** 2 +
                                      s_q4[:, :, :, self.simple_channels:] ** 2 +
                                      s_q5[:, :, :, self.simple_channels:] ** 2 +
                                      s_q6[:, :, :, self.simple_channels:] ** 2 +
                                      s_q7[:, :, :, self.simple_channels:] ** 2 +
                                      s_q8[:, :, :, self.simple_channels:] ** 2 +
                                      s_q9[:, :, :, self.simple_channels:] ** 2 +
                                      s_q10[:, :, :, self.simple_channels:] ** 2 +
                                      s_q11[:, :, :, self.simple_channels:] ** 2 +
                                      s_q12[:, :, :, self.simple_channels:] ** 2 +
                                      s_q13[:, :, :, self.simple_channels:] ** 2 +
                                      s_q14[:, :, :, self.simple_channels:] ** 2 +
                                      s_q15[:, :, :, self.simple_channels:] ** 2
                                     
                                     
                                     ) / np.sqrt(16))
        s = self.simple(s_q0[:, :, :, :self.simple_channels])

        return self.gabors(self.k_exc * tf.concat((s, c), axis=-1))

    def noise_f(self, inputs):
        x = inputs
        if self.noise_mode == 'neuronal':
            eps = 10e-5
            x *= self.noise_scale
            x += self.noise_level
            if self.fixed_noise is not None:
                x += self.fixed_noise * tf.math.sqrt(tf.nn.relu(x) + eps)
            else:
                x += tf.random.normal(tf.shape(x), stddev=1) * \
                    tf.math.sqrt(tf.nn.relu(x) + eps)
            x -= self.noise_level
            x /= self.noise_scale
        if self.noise_mode == 'gaussian':
            if self.fixed_noise is not None:
                x += self.fixed_noise * self.noise_scale
            else:
                x += tf.random.normal(tf.shape(x), stddev=1) * self.noise_scale
        return self.noise(x)

    def set_noise_mode(self, noise_mode=None, noise_scale=1, noise_level=1):
        self.noise_mode = noise_mode
        self.noise_scale = noise_scale
        self.noise_level = noise_level

    def fix_noise(self, batch_size=256, seed=None):
        if seed:
            tf.random.set_seed(seed)
        noise_mean = tf.zeros((batch_size, int(
            self.input_size / self.stride), int(self.input_size / self.stride), self.out_channels))
        if self.noise_mode:
            self.fixed_noise = tf.random.normal(noise_mean.shape, stddev=1)

    def unfix_noise(self):
        self.fixed_noise = None

        
        
        
        
        
        


def VOneNet_with_16_GFBs(sf_corr=0.75, sf_max=9, sf_min=0, rand_param=False, gabor_seed=0,
            simple_channels=256, complex_channels=256,
            noise_mode='neuronal', noise_scale=0.35, noise_level=0.07, k_exc=25,
            model_arch='resnet50', image_size=300, visual_degrees=8, ksize=25, stride=4, class_count=14):

    out_channels = simple_channels + complex_channels

    sf, theta, phase, nx, ny = generate_gabor_param(
        out_channels, gabor_seed, rand_param, sf_corr, sf_max, sf_min)

    gabor_params = {'simple_channels': simple_channels, 'complex_channels': complex_channels, 'rand_param': rand_param,
                    'gabor_seed': gabor_seed, 'sf_max': sf_max, 'sf_corr': sf_corr, 'sf': sf.copy(),
                    'theta': theta.copy(), 'phase': phase.copy(), 'nx': nx.copy(), 'ny': ny.copy()}
    arch_params = {'k_exc': k_exc, 'arch': model_arch,
                   'ksize': ksize, 'stride': stride}

    # Conversions
    ppd = image_size / visual_degrees

    sf = sf / ppd
    sigx = nx / sf
    sigy = ny / sf
    theta = theta/180 * np.pi
    phase = phase / 180 * np.pi

    vone_block = VOneBlock_16_GFBs(sf=sf, theta=theta, sigx=sigx, sigy=sigy, phase=phase,
                           k_exc=k_exc, noise_mode=noise_mode, noise_scale=noise_scale, noise_level=noise_level,
                           simple_channels=simple_channels, complex_channels=complex_channels,
                           ksize=ksize, stride=stride, input_size=image_size)
    
    
    
    
    img_shape=(image_size,image_size,3)
#     model_back_end=tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max')
    
#     inputs = tf.keras.Input(shape=(image_size, image_size, 3))
#     x = vone_block(inputs)
    
    
#     if model_arch:
    bottleneck = Conv2D(3, kernel_size=1, strides=1, use_bias=False,kernel_initializer=tf.keras.initializers.HeNormal())

#         # if model_arch.lower() == 'resnet50':
#         #     print('Model: ', 'VOneResnet50')
#         #     model_back_end = ResNetBackEnd(
#         #         block=Bottleneck, layers=[3, 4, 6, 3])
#         # elif model_arch.lower() == 'alexnet':
#         #     print('Model: ', 'VOneAlexNet')
#         #     model_back_end = AlexNetBackEnd()
#         # elif model_arch.lower() == 'cornets':
#         #     print('Model: ', 'VOneCORnet-S')
#         #     model_back_end = CORnetSBackEnd()
#         # TODO: change model_back_end to our block

    inputs = tf.keras.Input(shape=(image_size, image_size, 3))
    x = vone_block(inputs)
    x = bottleneck(x)
    
    
    # 获取模型后端的输入张量
#     backend_input = model_back_end.input

#     print(x.shape)
#     assert(False)
    
    
    model_back_end=tf.keras.applications.efficientnet.EfficientNetB0(include_top=False, weights="imagenet",input_shape=x.shape[1:], pooling='max')
    model_back_end.trainable = True
#     print(backend_input.shape)

    
    # 用 bottleneck 的输出替换模型后端的输入张量
    backend_output = model_back_end(x)
    
    
    x=backend_output
    x=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
    x=Dense(1024,activation='relu')(x)
    outputs=Dense(class_count, activation='sigmoid')(x)

    
#     outputs = model_back_end.output
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(Adamax(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy']) 
#     else:
#         print('Model: ', 'VOneNet')
#         model = vone_block

    model.image_size = image_size
    model.visual_degrees = visual_degrees
    model.gabor_parms = gabor_params
    model.arch_params = arch_params

    return model

# <center>Data Preprocessing</center>

In [ ]:
# This is the kaggle dataset path
datapath=r'../input/pizza-images-with-topping-labels/pizza_data/labels.csv'
imgpath=r'../input/pizza-images-with-topping-labels/pizza_data/images'

# datapath=r'pizza_data/labels.csv'
# imgpath=r'pizza_data/images'
df=pd.read_csv(datapath)
df=df.sample(n=9000, replace=False, random_state = 1)
df['plain']=0
columns=df.columns

df['image_name']=df['image_name'].apply(lambda x: os.path.join(imgpath,x))
df = df.rename(columns={'image_name': 'filepaths'})

for i in range (len(df)):
    label_list=[]
    for j in range (1,len(df.columns)-1):
        column=df.columns[j]        
        label=df.iloc[i][column]
        label_list.append(label)
    max=np.max(label_list)        
    if max == 0:         
        df['plain'].iloc[i]=1
train_df, dummy_df=train_test_split(df, train_size=.8, shuffle=True, random_state=123)
valid_df, test_df=train_test_split(dummy_df, train_size=.5, shuffle=True, random_state=123)
print('train_df length: ', len(train_df), '  test_df length: ', len(test_df), '  valid_df length: ', len(valid_df)) 

In [ ]:
img_size=(300,300)
columns=df.columns[1:]
class_count=len(columns)
generator=ImageDataGenerator()
train=generator.flow_from_dataframe(train_df, x_col='filepaths', y_col=columns, target_size=img_size, batch_size=30,shuffle=True,seed=123, class_mode='raw')
val=generator.flow_from_dataframe(valid_df, x_col='filepaths', y_col=columns, target_size=img_size, batch_size=30,shuffle=False,class_mode='raw')
test=generator.flow_from_dataframe(valid_df, x_col='filepaths', y_col=columns, target_size=img_size, batch_size=30,shuffle=False,class_mode='raw')

# <center>Display Sample Images</center>

In [ ]:
def display_samples(img, classes): 
    images,labels=next(img)
    plt.figure(figsize=(25, 25))
    num_of_images=10
    for i in range(num_of_images):        
        plt.subplot(5, 5, i + 1)
        image=images[i] /255 
        label=labels[i]
        title=''
        for j in range(len(label)):
            value=label[j]
            if value == 1:
                title=title + ' '+ classes[j]
        plt.imshow(image)
        fname=os.path.basename(img.filenames[i])        
        plt.title(title, fontsize=15)
    plt.show()
    
display_samples(train, columns )

# <center>Run & train the Model</center>

In [ ]:
lr=.001
# model=VOneNet()
# model=VOneNet_with_four_GFBs()
# model=VOneNet_with_eight_GFBs()
model=VOneNet_with_Only_Simple_GFBs()
# model=EfficientNet_B0_Dense1024(img_size, lr, class_count)

In [ ]:
epochs=50

In [ ]:
history=model.fit(x=train,  epochs=epochs, verbose=1, validation_data=val, shuffle=True,  initial_epoch=0)

In [ ]:
model.save_weights('/kaggle/working/ModelOutput')

# <center>Visualization \& Evaluation</center>

In [ ]:
def tr_plot(tr_data, start_epoch):
    tacc=tr_data.history['accuracy']
    tloss=tr_data.history['loss']
    vacc=tr_data.history['val_accuracy']
    vloss=tr_data.history['val_loss']
    Epoch_count=len(tacc)+ start_epoch
    Epochs=[]
    for i in range (start_epoch ,Epoch_count):
        Epochs.append(i+1)   
    index_loss=np.argmin(vloss)
    val_lowest=vloss[index_loss]
    index_acc=np.argmax(vacc)
    acc_highest=vacc[index_acc]
    plt.style.use('fivethirtyeight')
    sc_label='best epoch= '+ str(index_loss+1 +start_epoch)
    vc_label='best epoch= '+ str(index_acc + 1+ start_epoch)
    fig,axes=plt.subplots(nrows=1, ncols=2, figsize=(25,10))
    axes[0].plot(Epochs,tloss, 'r', label='Training loss')
    axes[0].plot(Epochs,vloss,'g',label='Validation loss' )
    axes[0].scatter(index_loss+1 +start_epoch,val_lowest, s=150, c= 'blue', label=sc_label)
    axes[0].scatter(Epochs, tloss, s=100, c='red')    
    axes[0].set_title('Training and Validation Loss')
    axes[0].set_xlabel('Epochs', fontsize=18)
    axes[0].set_ylabel('Loss', fontsize=18)
    axes[0].legend()
    axes[1].plot (Epochs,tacc,'r',label= 'Training Accuracy')
    axes[1].scatter(Epochs, tacc, s=100, c='red')
    axes[1].plot (Epochs,vacc,'g',label= 'Validation Accuracy')
    axes[1].scatter(index_acc+1 +start_epoch,acc_highest, s=150, c= 'blue', label=vc_label)
    axes[1].set_title('Training and Validation Accuracy')
    axes[1].set_xlabel('Epochs', fontsize=18)
    axes[1].set_ylabel('Accuracy', fontsize=18)
    axes[1].legend()
    plt.tight_layout    
    plt.show()
    return index_loss
    
loss_index=tr_plot(history,0)

In [ ]:
loss, accuracy=model.evaluate(test)
print (accuracy)

# <center>White Box Adversarial Attack</center>

In [ ]:
#load the model
lr = 0.01
# model = EfficientNet_B0_Dense1024(img_size, lr, class_count)
model = VOneNet()
epochs=15
# history=model.fit(x=train,  epochs=epochs, verbose=1, validation_data=val, shuffle=False,  initial_epoch=0)
model.load_weights('/kaggle/input/voneefficient/ModelOutput')
# model.save_weights('/kaggle/working/ModelOutput')
# !mkdir /kaggle/working/output
# !rm -rf /kaggle/working/*

In [ ]:
# Generate the FGSM attack
!rm -rf /kaggle/working/output
!mkdir /kaggle/working/output
import pandas as pd
import os
import tensorflow as tf
from keras.preprocessing.image import array_to_img
from PIL import Image

#Generate the label according to the labels.csv
def label_preprocess(filepath):
    mask = df["filepaths"] == filepath
    matching_rows = df[mask]
    if matching_rows.shape[0] == 0:
        print("Not found")
        return 
    else:
        label = []
        for i in range(1,15):
            label.append(matching_rows.iloc[0,i])
        return label
    
#Generate the adversarial attack images. input image is a Image object, output is a numpy
def fgsm_attack(image, epsilon, pred, model): 
    # Create a tensor from the input image
    img_tensor = tf.keras.preprocessing.image.img_to_array(image)
    if img_tensor.shape[-1] !=3:
        return None, False
    # Add a batch dimension
    img_tensor = np.expand_dims(img_tensor, axis=0)

    # Create a tensor from the input image
    input_tensor = tf.convert_to_tensor(img_tensor)

    # Watch the input tensor with GradientTape
    with tf.GradientTape() as tape:
        tape.watch(input_tensor)

        # Get the output of the model
        output = model(input_tensor)
        # Compute the loss
        loss = tf.keras.losses.binary_crossentropy(pred, output)

    # Get the gradient of the loss with respect to the input tensor
    gradient = tape.gradient(loss, input_tensor)

    # Get the sign of the gradient
    signed_gradient = tf.sign(gradient)

    # Add the sign of the gradient to the input tensor to generate the perturbation
    perturbed_tensor = input_tensor + epsilon * signed_gradient

    # Clip the perturbed tensor to ensure it stays within the valid pixel range
    perturbed_tensor = tf.clip_by_value(perturbed_tensor, 0, 255)

    # Convert the perturbed tensor back to an array
    perturbed_array = np.array(perturbed_tensor)

    # Remove the batch dimension
    perturbed_array = np.squeeze(perturbed_array, axis=0)
#     print(perturbed_array.shape)
    # Convert the array to a PIL Image
#     perturbed_image = Image.fromarray(np.int8(perturbed_array*255))

    return perturbed_array, True

def generate_adv(model, input_dir = '../input/pizza-images-with-topping-labels/pizza_data/images/', output_dir= '../working/output', eps = 0.1):
    # Loop through each file in the input directory
    i=0
    for filename in os.listdir(input_dir):
        # Check if the file is a .jpg image
        if filename.endswith('.jpg'):
            i+=1
            if i%10 == 0:
                print(i)
            if i==500:
                break
            # Load the image
            image_path = os.path.join(input_dir, filename)
    #         print(image_path)
            img = Image.open(image_path)
            img = img.resize((300,300))
            pred = label_preprocess(image_path)
            pred = np.expand_dims(pred, axis=0)
            # Generate an adversarial example with epsilon = 0.1
            adv_img, flag = fgsm_attack(img, eps, pred, model)
            if flag == False:
                print(f"Format incorrect: {filename}")
                continue
    #         print(adv_img.shape)
            adv_img = Image.fromarray(np.uint8(adv_img))
            # Save the adversarial example to disk
            adv_img.save('/kaggle/working/output/'+filename)

def main():
    #Hyperparameters
    lr = 0.001
    #Here indicate the model you want to attack
    model = EfficientNet_B0_Dense1024(img_size, lr, class_count)
    epochs=15
    #input directory
    input_dir = '../input/pizza-images-with-topping-labels/pizza_data/images/'
    #output directory
    output_dir = '../working/output'
    model.load_weights('/kaggle/input/efficientnet/ModelOutput')
    
    #generate adversarial images
    generate_adv(model, input_dir, output_dir, eps = 25.0)

main()

In [ ]:
#pack up
!tar -czvf vonenet-adv-eps5.tar.gz /kaggle/working/output